In [3]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import Range1d
from bokeh.io import export_png
output_notebook()

import numpy as np

from voltagebudget import util
from voltagebudget import neurons
from voltagebudget import plc
from voltagebudget.util import locate_firsts
from voltagebudget.util import locate_peaks
from voltagebudget.util import filter_voltages

# -
def create_traces(ts_x, changes, fn):
    d_variances = []
    errors = []
    for c in changes:
        initial, target, obs, ts_opt = fn(ts_x, c)

#         d_v = initial - obs 
        d_v = obs 
        err = util.mae(ts_x, ts_opt)

        d_variances.append(d_v)
        errors.append(err)

    d_variances = np.asarray(d_variances)
    errors = np.asarray(errors)

    return d_variances, errors

Loading BokehJS ...

In [4]:
t=0.5
stim_number=40
stim_onset=0.2
stim_offset=0.250
stim_rate=8
seed_stim=7525
time_step=1e-5

ns_x, ts_x = util.poisson_impulse(
        t,
        stim_onset,
        stim_offset - stim_onset,
        stim_rate,
        dt=time_step,
        n=stim_number,
        seed=seed_stim)

In [18]:
p = figure(plot_width=400, plot_height=200)
p.circle(ts_x, ns_x, color="grey", alpha=0.3)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.toolbar.logo = None
p.toolbar_location = None
show(p)

# Shared params

In [6]:
percent_change = 0.2
changes = np.linspace(0.0, percent_change, 100)

# Uniform

In [7]:
initial, target, obs, ts_opt = plc.uniform(ts_x, percent_change)

p = figure(plot_width=400, plot_height=200)
p.circle(ts_opt, ns_x, color="black")
p.circle(ts_x, ns_x, color="grey", alpha=0.3)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [8]:
# -
d_variances_u, errors_u = create_traces(ts_x, changes, plc.uniform)

p = figure(plot_width=400, plot_height=200)
p.line(x=d_variances_u, y=errors_u, color="black")
p.xaxis.axis_label = 'Delta Variance (ms)'
p.yaxis.axis_label = 'Error (ms)'
# p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# Extreme deviants

## Max

In [9]:
initial, target, obs, ts_opt = plc.max_deviant(ts_x, percent_change)

p = figure(plot_width=400, plot_height=200)
p.circle(ts_opt, ns_x, color="black")
p.circle(ts_x, ns_x, color="grey", alpha=0.3)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

print(initial, target, obs)

# Save for fig
X = np.vstack([ns_x, ts_x, ts_opt])
header = "n,ts,ts_opt"
np.savetxt("../analysis/plc_spikes.csv", X.T, delimiter=",", header=header, comments="")

(0.012855946336618894, 0.010284757069295115, 0.01028471176871509)


In [10]:
# -
d_variances_ma, errors_ma = create_traces(ts_x, changes, plc.max_deviant)

p = figure(plot_width=300, plot_height=200)
p.line(x=d_variances_ma, y=errors_ma, color="black")
p.circle(x=d_variances_ma, y=errors_ma, color="black")
p.xaxis.axis_label = 'Variance (ms)'
p.yaxis.axis_label = 'Error (ms)'
# p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# Save for fig
X = np.vstack([d_variances_ma, errors_ma, changes])
header = "variances,errors,percent_changes"
np.savetxt("../analysis/plc_traces.csv", X.T, delimiter=",", header=header, comments="")

## Left side deviants

In [11]:
initial, target, obs, ts_opt = plc.max_deviant(ts_x, percent_change, side='left')

p = figure(plot_width=400, plot_height=200)
p.circle(ts_opt, ns_x, color="black")
p.circle(ts_x, ns_x, color="grey", alpha=0.3)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

print(initial, target, obs)

# Save for fig
X = np.vstack([ns_x, ts_x, ts_opt])
header = "n,ts,ts_opt"
np.savetxt("../analysis/plc_spikes.csv", X.T, delimiter=",", header=header, comments="")

(0.012855946336618894, 0.010284757069295115, 0.01028434139834472)


In [12]:
# -
d_variances_left, errors_left = create_traces(ts_x, changes, 
                                          lambda x, p: plc.max_deviant(x, p, side='left'))

p = figure(plot_width=300, plot_height=200)
p.line(x=d_variances_ma, y=errors_ma, color="black")
p.circle(x=d_variances_ma, y=errors_ma, color="black")
p.xaxis.axis_label = 'Variance (ms)'
p.yaxis.axis_label = 'Error (ms)'
# p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# Save for fig
X = np.vstack([d_variances_ma, errors_ma, changes])
header = "variances,errors,percent_changes"
np.savetxt("../analysis/plc_traces.csv", X.T, delimiter=",", header=header, comments="")

# Compare traces

In [13]:
# -
p = figure(plot_width=300, plot_height=250)
p.line(y=d_variances_ma, x=errors_ma, color="black", legend="Max")
p.line(y=d_variances_left, x=errors_left, color="red", legend="Left")
p.line(y=d_variances_u, x=errors_u, color="orange", legend="Unif")
p.yaxis.axis_label = 'Variance (ms)'
p.xaxis.axis_label = 'Error (ms)'
# p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# CC

In [14]:
percent_change = .4

initial, target, obs, ts_opt = plc.coincidence(ts_x, percent_change)

p = figure(plot_width=400, plot_height=200)
p.circle(ts_opt, ns_x, color="black")
p.circle(ts_x, ns_x, color="grey", alpha=0.3)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [15]:
initial, target, obs

(0.012855946336618894, 0.007713567801971336, 0.004145474460953744)

In [16]:
# -
d_variances_cc, errors_cc = create_traces(ts_x, changes, plc.coincidence)

p = figure(plot_width=400, plot_height=200)
p.line(x=d_variances_cc, y=errors_cc, color="black")
p.circle(x=d_variances_cc, y=errors_cc, color="black")
p.xaxis.axis_label = 'Delta Variance (ms)'
p.yaxis.axis_label = 'Error (ms)'
# p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [17]:
errors_cc

array([0.        , 0.01100377, 0.01100377, 0.01100377, 0.01100377,
       0.01100377, 0.01100377, 0.01100377, 0.01100377, 0.01100377,
       0.01100377, 0.01100377, 0.01100377, 0.01100377, 0.01100377,
       0.01100377, 0.01100377, 0.01100377, 0.01100377, 0.01100377,
       0.01269884, 0.01269884, 0.01269884, 0.01269884, 0.01269884,
       0.01269884, 0.01269884, 0.01269884, 0.01269884, 0.01269884,
       0.01269884, 0.01269884, 0.01269884, 0.01269884, 0.01269884,
       0.01269884, 0.01269884, 0.01269884, 0.01269884, 0.01269884,
       0.01269884, 0.01269884, 0.01269884, 0.01269884, 0.01269884,
       0.01269884, 0.01269884, 0.01269884, 0.01269884, 0.01269884,
       0.01269884, 0.01269884, 0.01269884, 0.01269884, 0.01269884,
       0.01269884, 0.01269884, 0.01269884, 0.01269884, 0.01269884,
       0.01269884, 0.01269884, 0.01269884, 0.01269884, 0.01269884,
       0.01269884, 0.01269884, 0.01269884, 0.01269884, 0.01269884,
       0.01269884, 0.01269884, 0.01269884, 0.01269884, 0.01269